In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
output_dir = '/content/drive/MyDrive/Transcripts'

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

os.chdir(output_dir)

# Verify the current working directory
print(f"Current working directory: {os.getcwd()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/MyDrive/Transcripts


In [2]:
!pip install pdfplumber pandas requests beautifulsoup4 python-dateutil PyPDF2 -q

In [4]:
#@ This works

import re
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dateutil import parser
from urllib.parse import urljoin
import datetime
import os
from collections import defaultdict
import io
import PyPDF2
import pdfplumber

# Updated URLs and constants
FED_BASE_URL = "https://www.federalreserve.gov"
FOMC_HISTORICAL_BASE = "https://www.federalreserve.gov/monetarypolicy/fomchistorical"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/100.0.4896.127 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
}

# Define officials with their roles and tenures
officials_text = """Donald Kohn, Governor, 2002-2006
Donald Kohn, Vice Chair, 2006-2010
Alan Greenspan, Chair, 1987-2006
Ben Bernanke, Chair, 2006-2014
Janet Yellen, Vice Chair, 2010-2014
Janet Yellen, Chair, 2014-2018
Jerome Powell, Governor, 2012-2018
Jerome Powell, Chair, 2018-2024
Randall Kroszner, Governor, 2006-2009
Kevin Warsh, Governor, 2006-2011
Elizabeth Duke, Governor, 2008-2013
Daniel Tarullo, Governor, 2009-2017
Sarah Bloom Raskin, Governor, 2010-2014
Lael Brainard, Governor, 2014-2022
Lael Brainard, Governor, 2022-2023
Stanley Fischer, Vice Chair, 2014-2017
Jeremy Stein, Governor, 2012-2014
Michelle Bowman, Governor, 2018-2024
Richard Clarida, Vice Chair, 2018-2022
Randal Quarles, Governor, 2017-2021
Christopher Waller, Governor, 2020-2024
Michael Barr, Governor, 2022-2024
Lisa Cook, Governor, 2022-2024
Adriana Kugler, Governor, 2023-2025
Philip Jefferson, Governor, 2022-2023
Philip Jefferson, Vice Chair, 2023-2025
Susan S. Bies, Governor, 2001-2007
Mark W. Olson, Governor, 2001-2006
Roger W. Ferguson, Vice Chair, 1997-2006
Edward M. Gramlich, Governor, 1997-2005
Frederic S. Mishkin, Governor, 2006-2008
Cathy E. Minehan, President, Boston, 1994-2007
Eric S. Rosengren, Boston, 2007-2021
Susan M. Collins, Boston, 2022-2024
William McDonough, New York, 1993-2003
William C. Dudley, New York, 2009-2018
Timothy Geithner, New York, 2003-2009
John C. Williams, New York, 2011-2024
Edward G. Boehne, Philadelphia, 1981-2000
Anthony M. Santomero, Philadelphia, 2000-2006
Charles I. Plosser, Philadelphia, 2006-2015
Patrick T. Harker, Philadelphia, 2015-2024
Jerry Jordan, Cleveland, 1992-2003
Sandra Pianalto, Cleveland, 2003-2014
Loretta Mester, Cleveland, 2014-2024
Alfred Broaddus, Richmond, 1993-2004
Jeffrey M. Lacker, Richmond, 2004-2017
Thomas I. Barkin, Richmond, 2018-2024
Jack Guynn, Atlanta, 1996-2006
Dennis P. Lockhart, Atlanta, 2007-2017
Raphael Bostic, Atlanta, 2017-2024
Michael Moskow, Chicago, 1994-2007
Charles L. Evans, Chicago, 2007-2023
Austan D. Goolsbee, Chicago, 2023-2024
William Poole, St. Louis, 1998-2008
James Bullard, St. Louis, 2008-2023
Alberto G. Musalem, St. Louis, 2024-2024
Gary H. Stern, Minneapolis, 1985-2009
Narayana Kocherlakota, Minneapolis, 2009-2015
Neel Kashkari, Minneapolis, 2016-2024
Thomas M. Hoenig, Kansas City, 1991-2011
Esther L. George, Kansas City, 2011-2023
Jeffrey R. Schmid, Kansas City, 2023-2024
Robert D. McTeer, Dallas, 1991-2004
Richard W. Fisher, Dallas, 2005-2015
Robert Steven Kaplan, Dallas, 2015-2021
Robert T. Parry, San Francisco, 1986-2004
John Williams, San Francisco, 2004-2018
Mary C. Daly, San Francisco, 2018-2024
Lorie K. Logan, Dallas, 2022-2024"""

def parse_officials(officials_text):
    """Parse the officials text into a structured dictionary"""
    officials = []

    for line in officials_text.strip().split('\n'):
        line = line.strip()
        if not line:
            continue

        # Split on comma to get name, role, years
        parts = [p.strip() for p in line.split(',')]

        if len(parts) >= 3:
            name = parts[0]
            role = parts[1]
            years = parts[2]

            # Extract years
            year_match = re.search(r'(\d{4})-(\d{4})', years)
            if year_match:
                start_year = int(year_match.group(1))
                end_year = int(year_match.group(2))

                # Create name parts for matching
                name_parts = name.split()
                last_name = name_parts[-1].upper() if name_parts else ""
                first_name = name_parts[0].upper() if name_parts else ""

                officials.append({
                    'name': name,
                    'role': role,
                    'start_year': start_year,
                    'end_year': end_year,
                    'last_name': last_name,
                    'first_name': first_name,
                    'full_name_upper': name.upper()
                })

    print(f"DEBUG: Total officials parsed: {len(officials)}")
    return officials

def generate_transcript_id(official_name, date_str, sequence_num):
    """Generate transcript ID in format: FirstInitialLastName_YYYYMMDD_transcript_N"""
    name_parts = official_name.split()
    if len(name_parts) >= 2:
        first_initial = name_parts[0][0].upper()
        last_name = name_parts[-1]
        name_part = f"{first_initial}{last_name}"
    else:
        name_part = official_name.replace(' ', '')

    try:
        if isinstance(date_str, str):
            date_obj = parser.parse(date_str)
        else:
            date_obj = date_str
        date_part = date_obj.strftime('%Y%m%d')
    except:
        date_part = "00000000"

    return f"{name_part}_{date_part}_transcript_{sequence_num}"

def fetch_page(url, headers, max_retries=3, timeout=30):
    """Fetch a web page with retry logic"""
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            if response.ok:
                return response.content
            else:
                print(f"Failed to fetch {url}: Status code {response.status_code}")
                time.sleep(1)
        except (requests.RequestException, ConnectionError) as e:
            print(f"Attempt {attempt+1}/{max_retries} failed for {url}: {e}")
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)

    print(f"Request failed after multiple retries for {url}")
    return None

def find_transcript_pdfs_for_year(year):
    """Find all transcript PDF links for a given year - ONLY actual meeting transcripts"""
    transcript_links = []
    year_url = f"{FOMC_HISTORICAL_BASE}{year}.htm"

    print(f"Searching for transcripts on {year_url}")

    html_content = fetch_page(year_url, HEADERS)
    if not html_content:
        print(f"Could not fetch content for {year}")
        return transcript_links

    soup = BeautifulSoup(html_content, "html.parser")

    # Look for links to PDFs - ONLY meeting transcripts
    for link in soup.find_all('a'):
        href = link.get('href', '')
        text = link.text.lower()

        # ONLY process PDFs that are actual meeting transcripts
        # These typically have "meeting" in the filename
        if (href.lower().endswith('.pdf') and
            'meeting' in href.lower() and
            # Exclude supporting documents
            not any(doc_type in href.lower() for doc_type in
                   ['beige', 'bluebook', 'greenbook', 'gbpt', 'gbsup', 'agenda', 'material'])):

            if not href.startswith('http'):
                href = urljoin(FED_BASE_URL, href)

            meeting_date = extract_date_from_pdf_url(href)
            if meeting_date:
                transcript_links.append({
                    'url': href,
                    'date': meeting_date
                })
                print(f"Found MEETING transcript for {meeting_date.strftime('%Y-%m-%d')}: {href}")

    print(f"Total meeting transcripts found for {year}: {len(transcript_links)}")
    return transcript_links

def extract_date_from_pdf_url(url):
    """Extract meeting date from PDF URL with multiple patterns"""
    # Try FOMC20170614 format
    date_match = re.search(r'FOMC(\d{8})', url)
    if date_match:
        date_str = date_match.group(1)
        try:
            return datetime.datetime.strptime(date_str, '%Y%m%d')
        except ValueError:
            pass

    # Try other date formats in URL
    date_patterns = [
        r'(\d{4})(\d{2})(\d{2})',  # YYYYMMDD
        r'(\d{2})-(\d{2})-(\d{4})',  # MM-DD-YYYY
        r'(\d{4})-(\d{2})-(\d{2})',  # YYYY-MM-DD
    ]

    for pattern in date_patterns:
        match = re.search(pattern, url)
        if match:
            try:
                groups = match.groups()
                if len(groups) == 3:
                    if len(groups[0]) == 4:  # YYYY format
                        year, month, day = int(groups[0]), int(groups[1]), int(groups[2])
                    else:  # MM-DD-YYYY format
                        month, day, year = int(groups[0]), int(groups[1]), int(groups[2])
                    return datetime.datetime(year, month, day)
            except ValueError:
                continue

    # Extract year from URL path
    year_match = re.search(r'/(\d{4})/', url)
    if year_match:
        year = int(year_match.group(1))
        return datetime.datetime(year, 1, 1)

    return None

def extract_text_from_pdf(pdf_content):
    """Extract text from PDF using available libraries with proper encoding"""
    # Try pdfplumber first (more reliable)
    try:
        with pdfplumber.open(io.BytesIO(pdf_content)) as pdf:
            text = ""
            for page in pdf.pages:
                extracted = page.extract_text()
                if extracted:
                    text += extracted + "\n"
            return text
    except Exception as e:
        print(f"Error with pdfplumber extraction: {e}")

    # Fallback to PyPDF2 if pdfplumber failed
    try:
        pdf_file = io.BytesIO(pdf_content)
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"

        return text
    except Exception as e:
        print(f"Error extracting PDF text with PyPDF2: {e}")

    return None

def clean_extracted_text(text):
    """Clean extracted text to fix formatting issues and remove unwanted patterns"""
    if not text:
        return text

    # Fix common encoding issues
    text = text.replace('â€"', '—')  # em-dash
    text = text.replace('â€™', "'")  # apostrophe
    text = text.replace('â€œ', '"')  # left double quote
    text = text.replace('â€', '"')   # right double quote

    # Prevent line breaks in "U.S." by replacing with a non-breaking version temporarily
    text = re.sub(r'U\.S\.', 'U_S_', text)

    # Remove date and page patterns
    # Pattern for "December 10-11, 2019 125 of 252"
    text = re.sub(
        r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}(?:-\d{1,2})?,\s+\d{4}\s+\d+\s+of\s+\d+',
        '', text
    )
    # Pattern for "6/26-27/01 87"
    text = re.sub(
        r'\d{1,2}/\d{1,2}(?:-\d{1,2})?/\d{2}\s+\d+',
        '', text
    )

    # Add line break after periods followed by capital letters (likely new sentences)
    # Avoid breaking U.S. by checking it's not our placeholder
    text = re.sub(r'\.(?!\s*U_S_)([A-Z])', r'.\n\1', text)

    # Restore U.S. by replacing placeholder
    text = text.replace('U_S_', 'U.S.')

    # Add line breaks before common section headers
    text = re.sub(r'([a-z])([A-Z][A-Z][A-Z])', r'\1\n\2', text)

    # Clean up excessive whitespace
    text = re.sub(r'\n\s*\n', '\n\n', text)  # Multiple newlines to double
    text = re.sub(r' +', ' ', text)  # Multiple spaces to single

    return text.strip()

def extract_speakers_and_statements(transcript_text):
    """Extract speaker segments from transcript text - focuses on ALL CAPS names"""
    # Look for patterns where names appear in ALL CAPS
    # Common patterns in FOMC transcripts:
    # "MR. TARULLO." or "CHAIRMAN GREENSPAN." or just "FERGUSON."

    patterns = [
        # Pattern 1: Title + ALL CAPS NAME followed by period
        r'\n\s*((?:MR|MS|DR|CHAIRMAN|CHAIR|VICE CHAIRMAN|VICE CHAIR|GOVERNOR|PRESIDENT)\.?\s+[A-Z]+(?:\s+[A-Z]+)*)\.\s*\n',

        # Pattern 2: Just ALL CAPS NAME followed by period (common format)
        r'\n\s*([A-Z]{2,}(?:\s+[A-Z]{2,})*)\.\s*\n',

        # Pattern 3: ALL CAPS NAME followed by colon
        r'\n\s*([A-Z]{2,}(?:\s+[A-Z]{2,})*):?\s*\n',

        # Pattern 4: Inline speaker format (less common but sometimes used)
        r'\b((?:MR|MS|DR|CHAIRMAN|CHAIR|VICE CHAIRMAN|VICE CHAIR|GOVERNOR|PRESIDENT)\.?\s+[A-Z]{2,}(?:\s+[A-Z]{2,})*)\.\s',
    ]

    all_matches = []

    for i, pattern in enumerate(patterns):
        matches = re.finditer(pattern, transcript_text, re.MULTILINE)
        for match in matches:
            speaker_name = match.group(1).strip()

            # Filter out obvious non-names (common transcript artifacts)
            excluded_phrases = [
                'FEDERAL RESERVE', 'BOARD OF GOVERNORS', 'MEETING', 'TRANSCRIPT',
                'PAGE', 'APPENDIX', 'EXHIBIT', 'TABLE', 'CHART', 'FIGURE',
                'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE',
                'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER',
                'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY',
                'FOMC', 'GDP', 'CPI', 'NOTE', 'NOTES', 'ADDENDA'
            ]

            # Only include if it looks like a person's name
            if (len(speaker_name) > 1 and
                not any(phrase in speaker_name for phrase in excluded_phrases) and
                not speaker_name.isdigit()):

                all_matches.append((match.start(), speaker_name, f"Pattern_{i+1}"))

    # Sort by position and remove duplicates
    all_matches = sorted(list(set(all_matches)), key=lambda x: x[0])

    # Debug: Show what patterns we're finding
    print(f"DEBUG: Found {len(all_matches)} potential speakers using ALL CAPS patterns:")
    for _, speaker, pattern in all_matches[:10]:  # Show first 10
        print(f"  {pattern}: '{speaker}'")
    if len(all_matches) > 10:
        print(f"  ... and {len(all_matches) - 10} more")

    # Extract text segments
    speakers_text = []
    for i in range(len(all_matches)):
        start_pos, speaker, pattern_used = all_matches[i]

        # Find the actual start of the text (after the speaker name)
        speaker_end = transcript_text.find(speaker, start_pos) + len(speaker)
        text_start = speaker_end

        # Skip past punctuation and whitespace
        while text_start < len(transcript_text) and transcript_text[text_start] in '.:;\s\n':
            text_start += 1

        # End position is start of next speaker or end of text
        end_pos = all_matches[i+1][0] if i < len(all_matches) - 1 else len(transcript_text)

        text = transcript_text[text_start:end_pos].strip()

        # Only include if we have substantial text (filter very short segments)
        if speaker and text and len(text) > 20:  # At least 20 characters
            speakers_text.append((speaker, text))

    return speakers_text

def match_speaker_to_official(speaker, officials, meeting_date):
    """Enhanced speaker matching focused on capitalized names"""
    speaker = speaker.strip().upper()

    # Extract potential name from speaker string by removing common titles/prefixes
    cleaned_speaker = speaker

    # Remove common prefixes to get to the actual name
    prefixes_to_remove = [
        "CHAIRMAN ", "CHAIR ", "VICE CHAIRMAN ", "VICE CHAIR ",
        "GOVERNOR ", "PRESIDENT ", "MR. ", "MS. ", "DR. ", "THE "
    ]

    for prefix in prefixes_to_remove:
        if cleaned_speaker.startswith(prefix):
            cleaned_speaker = cleaned_speaker[len(prefix):].strip()
            break

    # Now try to match the cleaned name against officials
    for official in officials:
        if not (official['start_year'] <= meeting_date.year <= official['end_year']):
            continue

        official_last = official['last_name']
        official_first = official['first_name']
        official_full = official['full_name_upper']

        # Try different matching strategies:

        # 1. Exact full name match
        if cleaned_speaker == official_full.replace(',', '').replace('.', ''):
            print(f"DEBUG: Full name match '{speaker}' -> {official['name']}")
            return official

        # 2. Last name exact match
        if cleaned_speaker == official_last:
            print(f"DEBUG: Last name match '{speaker}' -> {official['name']}")
            return official

        # 3. First + Last name match (handle various formats)
        name_combinations = [
            f"{official_first} {official_last}",  # JEROME POWELL
            f"{official_first[0]} {official_last}" if official_first else "",  # J POWELL
            f"{official_first[0]}. {official_last}" if official_first else "",  # J. POWELL
        ]

        for combo in name_combinations:
            if combo and cleaned_speaker == combo:
                print(f"DEBUG: Name combination match '{speaker}' -> {official['name']} via '{combo}'")
                return official

        # 4. Handle compound last names or middle initials
        # Split the cleaned speaker and see if last word matches last name
        speaker_parts = cleaned_speaker.split()
        if len(speaker_parts) >= 2:
            speaker_last_word = speaker_parts[-1]
            if speaker_last_word == official_last:
                # Check if first part could match first name or initial
                speaker_first_part = speaker_parts[0]
                if (speaker_first_part == official_first or
                    speaker_first_part == official_first[0] if official_first else False):
                    print(f"DEBUG: Multi-part name match '{speaker}' -> {official['name']}")
                    return official

    # 5. Special handling for some common transcript formats
    # Sometimes names appear with periods: "J.POWELL" or "JEROME.H.POWELL"
    if '.' in cleaned_speaker:
        # Remove periods and try again
        no_periods = cleaned_speaker.replace('.', ' ').strip()
        # Recursive call with cleaned version
        result = match_speaker_to_official(no_periods, officials, meeting_date)
        if result:
            return result

    return None

def process_transcripts(from_year=2000, to_year=2024, skip_existing=True):
    """Process all transcripts and extract speeches by officials"""
    officials = parse_officials(officials_text)

    os.makedirs('official_transcripts', exist_ok=True)

    # Check for existing files if skip_existing is True
    existing_officials = set()
    if skip_existing:
        print("Checking for existing official files...")
        for official in officials:
            filename = official['name'].replace(' ', '_').replace(',', '').replace('.', '')
            output_path = f"official_transcripts/{filename}.csv"
            if os.path.exists(output_path):
                existing_officials.add(official['name'])
                print(f"  ✓ Found existing file for {official['name']} - will skip")

        if existing_officials:
            print(f"\nSkipping {len(existing_officials)} officials with existing files")
            print("Set skip_existing=False to re-scrape all officials\n")
        else:
            print("No existing files found - will scrape all officials\n")

    official_data = defaultdict(list)
    official_date_sequences = defaultdict(lambda: defaultdict(int))

    # Statistics tracking
    total_speakers_found = 0
    matched_speakers = 0
    unmatched_speakers = set()

    for year in range(from_year, to_year + 1):
        if year > datetime.datetime.now().year:
            print(f"Skipping future year {year}")
            continue

        print(f"\nProcessing year {year}...")

        transcript_pdfs = find_transcript_pdfs_for_year(year)

        for transcript in transcript_pdfs:
            pdf_url = transcript['url']
            meeting_date = transcript['date']

            print(f"Processing transcript from {meeting_date.strftime('%Y-%m-%d')}: {pdf_url}")

            pdf_content = fetch_page(pdf_url, HEADERS)
            if not pdf_content:
                continue

            transcript_text = extract_text_from_pdf(pdf_content)
            if not transcript_text:
                continue

            speaker_segments = extract_speakers_and_statements(transcript_text)
            print(f"Found {len(speaker_segments)} speaker segments")

            for speaker, text in speaker_segments:
                total_speakers_found += 1

                matched_official = match_speaker_to_official(speaker, officials, meeting_date)

                if matched_official:
                    official_name = matched_official['name']

                    # Skip if we already have a file for this official and skip_existing is True
                    if skip_existing and official_name in existing_officials:
                        continue

                    matched_speakers += 1
                    official_name = matched_official['name']

                    date_key = meeting_date.strftime('%Y-%m-%d')
                    official_date_sequences[official_name][date_key] += 1
                    sequence_num = official_date_sequences[official_name][date_key]

                    transcript_id = generate_transcript_id(official_name, meeting_date, sequence_num)

                    official_data[official_name].append({
                        'id': transcript_id,
                        'Date': meeting_date.strftime('%Y-%m-%d'),
                        'Name': official_name,
                        'Role': matched_official['role'],
                        'Original_Speaker': speaker,
                        'Text': text,
                        'Transcript_URL': pdf_url
                    })

                    print(f"✓ Matched: {speaker} -> {official_name}")
                else:
                    unmatched_speakers.add(speaker)
                    print(f"✗ No match for: {speaker}")

            time.sleep(2)  # Rate limiting

    # Save results
    for official_name, entries in official_data.items():
      if entries:
          # Clean text in each entry
          for entry in entries:
              entry['Text'] = clean_extracted_text(entry['Text'])

          df = pd.DataFrame(entries)
          filename = official_name.replace(' ', '_').replace(',', '').replace('.', '')
          output_path = f"official_transcripts/{filename}.csv"

          # Specify UTF-8 encoding and escape characters properly
          df.to_csv(output_path, index=False, encoding='utf-8', escapechar='\\')
          print(f"Saved {len(entries)} entries for {official_name} to {output_path}")

    # Print statistics
    print(f"\n=== MATCHING STATISTICS ===")
    print(f"Total speaker segments found: {total_speakers_found}")
    print(f"Successfully matched: {matched_speakers}")
    print(f"Unmatched: {len(unmatched_speakers)}")
    print(f"Match rate: {matched_speakers/total_speakers_found*100:.1f}%")

    if unmatched_speakers:
        print(f"\nTop unmatched speakers:")
        for speaker in sorted(list(unmatched_speakers))[:20]:
            print(f"  - {speaker}")

    print(f"\nFiles created for {len(official_data)} officials")
    return official_data

def main():
    from_year = 2000
    to_year = 2025

    # Toggle to skip officials that already have files
    skip_existing_files = False  # Set to False to re-scrape all officials

    print(f"Extracting FOMC transcript data for officials from {from_year} to {to_year}")
    if skip_existing_files:
        print("Will skip officials with existing files (set skip_existing_files=False to override)")
    else:
        print("Will re-scrape ALL officials, including those with existing files")

    process_transcripts(from_year, to_year, skip_existing=skip_existing_files)

if __name__ == "__main__":
    main()

<>:397: SyntaxWarning: invalid escape sequence '\s'
<>:397: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-782585164.py:397: SyntaxWarning: invalid escape sequence '\s'
  while text_start < len(transcript_text) and transcript_text[text_start] in '.:;\s\n':


Streaming output truncated to the last 5000 lines.
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
DEBUG: Last name match 'CHAIRMAN POWELL' -> Jerome Powell
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
DEBUG: Last name match 'MS. MESTER' -> Loretta Mester
✓ Matched: MS. MESTER -> Loretta Mester
DEBUG: Last name match 'CHAIRMAN POWELL' -> Jerome Powell
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
DEBUG: Last name match 'MR. ROSENGREN' -> Eric S. Rosengren
✓ Matched: MR. ROSENGREN -> Eric S. Rosengren
DEBUG: Last name match 'CHAIRMAN POWELL' -> Jerome Powell
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
✗ No match for: MR. GOULD
DEBUG: Last name match 'CHAIRMAN POWELL' -> Jerome Powell
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
DEBUG: Last name match 'MR. HARKER' -> Patrick T. Harker
✓ Matched: MR. HARKER -> Patrick T. Harker
DEBUG: Last name match 'CHAIRMAN POWELL' -> Jerome Powell
✓ Matched: CHAIRMAN POWELL -> Jerome Powell
DEBUG: Last name match 'MR. KAPLAN' -> Robert Steven Kaplan
✓ Matched

In [5]:
import pandas as pd
import os
from collections import defaultdict

def clean_duplicate_transcripts(input_dir='official_transcripts', output_dir='cleaned_transcripts'):
    """
    Aggregates multiple transcript entries for the same official on the same date
    into a single entry by concatenating the text fields.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Get list of all CSV files in the input directory
    csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

    print(f"Found {len(csv_files)} CSV files to process")

    for csv_file in csv_files:
        input_path = os.path.join(input_dir, csv_file)
        print(f"Processing {csv_file}...")

        # Read the CSV file
        try:
            df = pd.read_csv(input_path, encoding='utf-8')
        except Exception as e:
            print(f"Error reading {csv_file}: {e}")
            continue

        if df.empty:
            print(f"  Skipping empty file: {csv_file}")
            continue

        # Ensure required columns are present
        required_columns = ['id', 'Date', 'Name', 'Role', 'Text', 'Transcript_URL']
        if not all(col in df.columns for col in required_columns):
            print(f"  Skipping {csv_file}: Missing required columns")
            continue

        # Group by Name and Date, aggregating other fields
        aggregated_data = []
        grouped = df.groupby(['Name', 'Date'])

        for (name, date), group in grouped:
            # Concatenate all text entries for this official-date pair
            combined_text = ' '.join(group['Text'].astype(str))

            # Take the first entry's values for other fields
            first_entry = group.iloc[0]

            # Generate a new ID without sequence number (e.g., ABroaddus_20000202_transcript)
            name_parts = name.split()
            if len(name_parts) >= 2:
                first_initial = name_parts[0][0].upper()
                last_name = name_parts[-1]
                name_part = f"{first_initial}{last_name}"
            else:
                name_part = name.replace(' ', '')
            date_part = date.replace('-', '')
            new_id = f"{name_part}_{date_part}_transcript"

            # Create the aggregated entry
            aggregated_data.append({
                'id': new_id,
                'Date': date,
                'Name': name,
                'Role': first_entry['Role'],
                'Original_Speaker': first_entry['Original_Speaker'],
                'Text': combined_text,
                'Transcript_URL': first_entry['Transcript_URL']
            })

        # Create a new DataFrame with aggregated data
        cleaned_df = pd.DataFrame(aggregated_data)

        # Save to new CSV in output directory
        output_filename = csv_file  # Keep the same filename
        output_path = os.path.join(output_dir, output_filename)
        cleaned_df.to_csv(output_path, index=False, encoding='utf-8', escapechar='\\')
        print(f"  Saved cleaned data to {output_path}: {len(cleaned_df)} entries (reduced from {len(df)})")

    print("\nCleaning complete!")

if __name__ == "__main__":
    clean_duplicate_transcripts()

Found 54 CSV files to process
Processing Alan_Greenspan.csv...
  Saved cleaned data to cleaned_transcripts/Alan_Greenspan.csv: 50 entries (reduced from 2750)
Processing Roger_W_Ferguson.csv...
  Saved cleaned data to cleaned_transcripts/Roger_W_Ferguson.csv: 50 entries (reduced from 401)
Processing William_McDonough.csv...
  Saved cleaned data to cleaned_transcripts/William_McDonough.csv: 27 entries (reduced from 271)
Processing Alfred_Broaddus.csv...
  Saved cleaned data to cleaned_transcripts/Alfred_Broaddus.csv: 36 entries (reduced from 177)
Processing Edward_G_Boehne.csv...
  Saved cleaned data to cleaned_transcripts/Edward_G_Boehne.csv: 2 entries (reduced from 11)
Processing Jerry_Jordan.csv...
  Saved cleaned data to cleaned_transcripts/Jerry_Jordan.csv: 23 entries (reduced from 159)
Processing Robert_T_Parry.csv...
  Saved cleaned data to cleaned_transcripts/Robert_T_Parry.csv: 35 entries (reduced from 275)
Processing Gary_H_Stern.csv...
  Saved cleaned data to cleaned_transcrip

In [ ]:
import os
import pandas as pd
import re
import numpy as np
import glob
from collections import Counter

# Define the labor market indicators and their related keywords
INDICATORS = {
    "Employment": ["employed", "employment", "employment-to-population", "employment to population", "employment/population", "e/p ratio", "employment-population ratio"],
    "Unemployment": ["unemployment", "unemployed", "jobless", "joblessness", "u-3", "u3 rate"],
    "Participation": ["participation", "marginally attached", "discouraged workers", "retirement", "labor force", "want a job", "labor force participation", "participation rate", "lfpr", "labor market participation"],
    "Wages": ["pay", "eci", "labor cost", "labor costs", "wage", "wages", "wage growth", "wage inflation", "compensation", "earnings", "salary", "salaries"],
    "Vacancies": ["vacancy", "vacancies", "job opening", "job openings", "jolts", "unfilled position", "open position", "vacant job", "vacancy rate", "job vacancy", "posted opening", "help wanted", "available job", "job posting", "position opening", "vacancy yield", "beveridge curve", "job advertisement", "job availability", "hiring difficulty"],
    "Quits": ["quit", "quits", "resignation", "resignations", "voluntary separation", "turnover", "voluntary turnover", "job changing", "job switching", "job hopping", "job-to-job"],
    "Layoffs": ["layoff", "layoffs", "involuntary separation", "dismissal", "dismissals", "discharged", "discharge", "downsizing", "redundancy", "workforce reduction", "job cuts", "reduction in force", "rif", "termination", "fired", "job loss", "permanent layoff", "temporary layoff", "furlough", "mass layoff", "involuntary job loss", "job destruction", "establishment closure", "job shedding"],
    "Hiring": ["hiring", "job finding", "payroll", "nonfarm payroll", "job gain", "job growth", "employment gain", "employment growth", "job creation"]
}

def get_labor_market_keywords():
    """Create a list of all labor market related keywords"""
    labor_keywords = ["labor market", "job market", "employment", "unemployment", "labor",
                    "labor markets", "job markets", "hiring", "workers", "unemployment",
                    "employment situation", "labor force", "job", "jobs"]

    # Add all indicator keywords to the labor market keywords
    for keywords in INDICATORS.values():
        labor_keywords.extend(keywords)

    return labor_keywords

def split_into_sentences(text):
    """Split text into sentences using regex"""
    # First replace common abbreviations to avoid splitting them incorrectly
    text = re.sub(r'(\b[A-Z])\. ', r'\1@@ ', text)  # Handle abbreviations like U.S.
    text = re.sub(r'(\b[A-Z][a-z])\. ', r'\1@@ ', text)  # Handle Dr., Mr., etc.

    # Split by sentence boundaries
    sentences = re.split(r'(?<=[.!?])\s+', text)

    # Restore the periods in abbreviations
    sentences = [re.sub(r'@@', '.', s) for s in sentences]

    return sentences

def get_indicator_counts(text):
    """Get counts of each indicator's keywords in the text"""
    text_lower = text.lower()

    indicator_counts = {indicator: 0 for indicator in INDICATORS}

    for indicator, keywords in INDICATORS.items():
        for keyword in keywords:
            # Use word boundaries to avoid partial matches
            matches = re.findall(r'\b' + re.escape(keyword.lower()) + r'\b', text_lower)
            count = len(matches)

            # Special handling for "employment" - exclude "maximum employment"
            if keyword.lower() == "employment":
                # Count how many times "maximum employment" appears and subtract from employment count
                max_employment_count = len(re.findall(r'\bmaximum employment\b', text_lower))
                count = max(0, count - max_employment_count)

            indicator_counts[indicator] += count

    return indicator_counts

def contains_labor_market_keyword(sentence, labor_keywords):
    """Check if a sentence contains any labor market related keywords"""
    sentence_lower = sentence.lower()

    for keyword in labor_keywords:
        if re.search(r'\b' + re.escape(keyword.lower()) + r'\b', sentence_lower):
            return True

    return False

def analyze_official_transcripts():
    """Analyze the transcripts of officials for labor market content"""
    # Get the list of all CSV files in the official_transcripts folder
    transcript_files = glob.glob("official_transcripts/*.csv")

    if not transcript_files:
        print("No transcript files found in 'official_transcripts' folder.")
        return None

    # Get the list of labor market keywords
    labor_keywords = get_labor_market_keywords()

    # List to store all results
    all_results = []

    # Process each official's transcript file
    for transcript_file in transcript_files:
        official_name = os.path.basename(transcript_file).replace('.csv', '').replace('_', ' ')
        print(f"Processing transcripts for {official_name}")

        try:
            # Load the transcript data
            official_df = pd.read_csv(transcript_file)

            # Process each transcript entry
            for _, row in official_df.iterrows():
                date = row['Date']
                text = row['Text']

                # Skip if text is missing
                if pd.isna(text) or text == "":
                    continue

                # Split the text into sentences
                sentences = split_into_sentences(text)

                # Identify labor market related sentences
                labor_sentences = []
                for sentence in sentences:
                    if contains_labor_market_keyword(sentence, labor_keywords):
                        labor_sentences.append(sentence)

                # Calculate percentage of speech focused on labor markets
                total_sentences = len(sentences)
                labor_sentences_count = len(labor_sentences)

                if total_sentences > 0:
                    labor_market_percentage = (labor_sentences_count / total_sentences) * 100
                else:
                    labor_market_percentage = 0

                # Combine all labor market sentences
                labor_market_text = " ".join(labor_sentences)

                # Get indicator counts for the labor market portion
                indicator_counts = get_indicator_counts(labor_market_text)

                # Calculate percentages for each indicator (reweighted to sum to 100%)
                total_indicator_mentions = sum(indicator_counts.values())

                indicator_percentages = {}
                if total_indicator_mentions > 0:
                    for indicator, count in indicator_counts.items():
                        indicator_percentages[indicator] = (count / total_indicator_mentions) * 100
                else:
                    indicator_percentages = {indicator: 0 for indicator in INDICATORS}

                # Create result record
                result = {
                    'Official_Name': official_name,
                    'Date': date,
                    'Labor_Market_Percentage': labor_market_percentage,
                    'Labor_Sentences': labor_sentences_count,
                    'Total_Sentences': total_sentences
                }

                # Add indicator percentages
                for indicator, percentage in indicator_percentages.items():
                    result[f'{indicator}_Percentage'] = percentage

                all_results.append(result)

        except Exception as e:
            print(f"Error processing {transcript_file}: {e}")

    # Convert to DataFrame
    if all_results:
        results_df = pd.DataFrame(all_results)

        # Convert date to datetime
        results_df['Date'] = pd.to_datetime(results_df['Date'])

        # Sort by official name and date
        results_df = results_df.sort_values(['Official_Name', 'Date'])

        # Add verification column that percentages sum to 100%
        indicator_columns = [f'{indicator}_Percentage' for indicator in INDICATORS]
        results_df['Sum_Indicator_Percentages'] = results_df[indicator_columns].sum(axis=1)

        # Save to CSV
        results_df.to_csv('official_labor_market_analysis.csv', index=False, encoding='utf-8')
        print(f"Analysis completed. {len(results_df)} transcript entries analyzed and saved to official_labor_market_analysis.csv")

        return results_df
    else:
        print("No results generated.")
        return None

def generate_official_summaries(results_df):
    """Generate summary statistics for each official"""
    if results_df is None or results_df.empty:
        print("No results to summarize.")
        return

    # Group by official name
    grouped = results_df.groupby('Official_Name')

    # Create summary dataframe
    summaries = []

    for official_name, group in grouped:
        # Calculate averages
        avg_labor_percentage = group['Labor_Market_Percentage'].mean()
        total_entries = len(group)
        labor_focused_entries = sum(group['Labor_Market_Percentage'] > 0)

        # Calculate average indicator percentages
        indicator_averages = {}
        for indicator in INDICATORS:
            col = f'{indicator}_Percentage'
            # Only consider entries where labor market was discussed
            labor_entries = group[group['Labor_Market_Percentage'] > 0]
            if not labor_entries.empty:
                indicator_averages[indicator] = labor_entries[col].mean()
            else:
                indicator_averages[indicator] = 0

        # Create summary record
        summary = {
            'Official_Name': official_name,
            'Total_Entries': total_entries,
            'Labor_Focused_Entries': labor_focused_entries,
            'Avg_Labor_Market_Percentage': avg_labor_percentage
        }

        # Add indicator averages
        for indicator, avg in indicator_averages.items():
            summary[f'Avg_{indicator}_Percentage'] = avg

        summaries.append(summary)

    # Convert to DataFrame
    summaries_df = pd.DataFrame(summaries)

    # Sort by average labor market percentage (descending)
    summaries_df = summaries_df.sort_values('Avg_Labor_Market_Percentage', ascending=False)

    # Save to CSV
    summaries_df.to_csv('official_labor_market_summaries.csv', index=False, encoding='utf-8')
    print(f"Generated summaries for {len(summaries_df)} officials saved to official_labor_market_summaries.csv")

    return summaries_df

def filter_labor_market_transcripts(results_df, min_percentage=0):
    """Filter to only include transcript entries that discuss labor markets"""
    if results_df is None or results_df.empty:
        print("No results to filter.")
        return None

    # Filter for entries that have labor market content
    filtered_df = results_df[results_df['Labor_Market_Percentage'] > min_percentage].copy()

    # Reset index
    filtered_df = filtered_df.reset_index(drop=True)

    # Filter out entries with no labor market content
    print("\nFiltering out entries with no labor market content...")
    results_df = results_df[results_df['Labor_Market_Percentage'] > 0]
    results_df = results_df.reset_index(drop=True)
    print(f"After filtering: {len(results_df)} entries with labor market content retained")

    # Now save the filtered results
    results_df.to_csv('labor_market_analysis.csv', index=False)

    return filtered_df

def main():
    """Main function to run the analysis"""
    print("Analyzing official transcripts for labor market content...")

    # Run the analysis
    results_df = analyze_official_transcripts()

    if results_df is not None:
        # Generate official summaries
        generate_official_summaries(results_df)

        # Filter to only include entries with labor market content
        filter_labor_market_transcripts(results_df, min_percentage=0)

        # Print overall statistics
        print("\nOverall Statistics:")
        print(f"Total transcript entries analyzed: {len(results_df)}")

        labor_entries = results_df[results_df['Labor_Market_Percentage'] > 0]
        print(f"Entries with labor market content: {len(labor_entries)} ({len(labor_entries)/len(results_df)*100:.2f}%)")

        print(f"Average percentage of transcript about labor markets: {results_df['Labor_Market_Percentage'].mean():.2f}%")

        # Calculate average percentages for each indicator (only for entries with labor content)
        if not labor_entries.empty:
            print("\nAverage Indicator Percentages (for entries with labor market content):")
            for indicator in INDICATORS:
                col = f'{indicator}_Percentage'
                avg_percentage = labor_entries[col].mean()
                print(f"Average for {indicator}: {avg_percentage:.2f}%")

if __name__ == "__main__":
    main()

Analyzing official transcripts for labor market content...
Processing transcripts for James Bullard
Processing transcripts for Charles L Evans
Processing transcripts for Richard W Fisher
Processing transcripts for Dennis P Lockhart
Processing transcripts for Jeffrey M Lacker
Processing transcripts for Sandra Pianalto
Processing transcripts for Thomas M Hoenig
Processing transcripts for William Poole
Processing transcripts for Gary H Stern
Processing transcripts for Charles I Plosser
Processing transcripts for Eric S Rosengren
Processing transcripts for William C Dudley
Processing transcripts for Narayana Kocherlakota
Processing transcripts for Janet Yellen
Processing transcripts for John C Williams
Processing transcripts for Esther L George
Processing transcripts for Jerome Powell
Processing transcripts for Loretta Mester
Processing transcripts for Patrick T Harker
Processing transcripts for Robert Steven Kaplan
Processing transcripts for Neel Kashkari
Processing transcripts for Raphae